## Tibetan convective systems 

Task: Filter out only convective cells which are (at least partly) over the TP 





In [3]:
import numpy as np 
import xarray as xr 
import warnings
warnings.simplefilter(action = "ignore", category = RuntimeWarning)

In [5]:
## elevation mask 


dem = '/media/juli/Data/projects/data/elevation/elevation_600x350.nc'
elevations = xr.open_dataarray(dem)
 
# mask as coordinates 
dem_mask = elevations.where(elevations >= 3000)
dem_mask.coords['mask'] = (('lon', 'lat'), dem_mask)


In [ ]:


removed = 0 

# loop through cells 
for cell in np.unique(tracks.cell.values):
    subset = tracks[tracks.cell == cell]
    print('checking heavy rain cores for cell:', cell, subset.shape)
    tp_flag = 0 
    # loop through timesteps of features for specific cell 
    for f in subset.frame.values: 
        # idx is the timestep index for respective timestep or mask file 
        idx = f
    
        # open corresponding precip and mask file 
        year = subset.time.values[0].year 
        month = subset.time.values[0].month
        if len(str(month))== 1: 
            month= '0' + str(month)
            
        # check whether segmented feature is in area above 3000 m 
        maskfile = '/media/juli/Data/projects/data/satellite_data/ncep/ctt/Save/tbbtracking/Mask_Segmentation_'+str(year) + str(month) + '.nc
        mask = xr.open_dataarray(maskfile)

 
        # get right timestep frames 
        seg= mask[idx,:,:]

        # get feature ID for frame 
        featureid= subset.feature[subset.frame== f].values[0]
        
        # create mask as coordinates 
        seg_mask = seg.where(seg == featureid)
        seg_mask.coords['mask'] = (('lat', 'lon'), seg_mask)
        # apply mask on precip data to extract precip values for feature in cell 
        elevation_values = dem_mask.where(seg_mask.coords['mask'].values > 1)
        arr= elevation_values.values.flatten()
        values = arr[~np.isnan(arr)]
          
        mountain_features = values[values >=3000].shape[0]
        if rain_features >= 1: 
            tp_flag += mountain_features

    if tp_flag ==  0: 
        # remove corresponding cell from track dataframe 
        tracks = tracks.drop(tracks[tracks.cell == cell].index)
        print(cell, ' removed.')
        removed += 1 
    else:
        print('heavy rain core present in:  ', cell, rain_features)
    
tracks.to_hdf(os.path.join(savedir,'Tracks_2001_TPflag.h5'),'table')    